<a href="https://colab.research.google.com/github/springboardmentor345a-create/Projects_2/blob/Harshdeep-Singh/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import os

# Upload the .pkl model files from your local machine
uploaded = files.upload()

# You can verify the uploaded files by listing the current directory content
print("Files uploaded:", uploaded.keys())
print("Files in current directory:", os.listdir('.'))

Saving To predict total points team can score.pkl to To predict total points team can score.pkl
Saving team league winner or not_LogisticRegression.pkl to team league winner or not_LogisticRegression.pkl
Saving Match winner.pkl to Match winner.pkl
Saving Goals_Assist_model.pkl to Goals_Assist_model.pkl
Files uploaded: dict_keys(['To predict total points team can score.pkl', 'team league winner or not_LogisticRegression.pkl', 'Match winner.pkl', 'Goals_Assist_model.pkl'])
Files in current directory: ['.config', 'To predict total points team can score.pkl', 'Match winner.pkl', 'team league winner or not_LogisticRegression.pkl', 'Goals_Assist_model.pkl', 'sample_data']


In [5]:
!pip install streamlit

In [6]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle
import os

st.set_page_config(page_title="EPL Predictions Hub", layout="wide")

# Function to safely load a model
def load_model(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                model = pickle.load(f)
            return model
        except Exception as e:
            st.error(f"Error loading model {filepath}: {e}")
            return None
    else:
        st.warning(f"Model file not found: {filepath}")
        return None

# --- Main Home Page ---
def home_page():
    st.title("⚽ EPL Predictions Hub")
    st.markdown("""
    Welcome to the English Premier League (EPL) Prediction application.
    Use the sidebar navigation to access different prediction models:
    - **Match Winner:** Predicts the outcome of a single match.
    - **Total Team Points:** Estimates total points a team can score in a season.
    - **Team League Winner:** Predicts if a team will win the league title.
    - **Goals & Assists:** Predicts player performance metrics.
    """)

# --- Model 1: Match Winner Prediction ---
def match_winner_page():
    st.title("🏆 Match Winner Prediction")
    st.markdown("Enter team and performance statistics to predict a match outcome.")

    # Load Model (place this inside function for single-file app structure)
    model = load_model('Match winner.pkl')
    if model is None:
        st.stop()

    with st.form("match_winner_form"):
        col1, col2 = st.columns(2)
        with col1:
            home_team = st.text_input("Home Team", "Arsenal")
            ht_form_pts = st.number_input("Home Team Form Pts (HTFormPts)", min_value=0, value=7)
            ht_gd = st.number_input("Home Team Goal Difference (HTGD)", value=2)
            diff_pts = st.number_input("Points Difference (DiffPts)", value=3)

        with col2:
            away_team = st.text_input("Away Team", "Man United")
            at_form_pts = st.number_input("Away Team Form Pts (ATFormPts)", min_value=0, value=5)
            at_gd = st.number_input("Away Team Goal Difference (ATGD)", value=-1)
            diff_form_pts = st.number_input("Form Points Difference (DiffFormPts)", value=2)

        submitted = st.form_submit_button("Predict Match Winner")

    if submitted:
        input_data = pd.DataFrame([{
            "HomeTeam": home_team,
            "AwayTeam": away_team,
            "HTFormPts": ht_form_pts,
            "ATFormPts": at_form_pts,
            "HTGD": ht_gd,
            "ATGD": at_gd,
            "DiffPts": diff_pts,
            "DiffFormPts": diff_form_pts
        }])

        # Note: Your model might expect features in a specific order or format (e.g., one-hot encoded teams).
        # This example assumes numerical features can be passed directly to the model's predict method.
        try:
            # Drop string columns before prediction if the model only takes numerical inputs
            features_for_pred = input_data.drop(columns=['HomeTeam', 'AwayTeam'])
            prediction = model.predict(features_for_pred)
            st.success(f"Prediction: {prediction[0]}")
        except Exception as e:
            st.error(f"An error occurred during prediction. Check your model's expected input format. Error: {e}")

# --- Model 2: To predict total points team can score ---
def total_points_page():
    st.title("📊 Total Team Points Prediction")
    st.markdown("Predict the total points a team can score in a season based on performance metrics.")

    model_path = 'To predict total points team can score.pkl'
    model = load_model(model_path)
    if model is None:
        st.stop()

    with st.form("total_points_form"):
        matches_played = st.number_input("Matches Played", min_value=0, value=38)
        goals_scored = st.number_input("Goals Scored", min_value=0, value=75)
        goals_conceded = st.number_input("Goals Conceded", min_value=0, value=35)

        submitted = st.form_submit_button("Predict Total Points")

    if submitted:
        input_data = pd.DataFrame([{
            'matches_played': matches_played,
            'goals_scored': goals_scored,
            'goals_conceded': goals_conceded
        }])
        try:
            prediction = model.predict(input_data)
            st.success(f"Predicted Total Points: {prediction[0]:.2f}")
        except Exception as e:
            st.error(f"An error occurred during prediction: {e}")

# --- Model 3: Team league winner or not_LogisticRegression ---
def league_winner_page():
    st.title("🏆 League Winner Prediction")
    st.markdown("Predict whether a team will win the league title using performance stats and target points.")

    model_path = 'team league winner or not_LogisticRegression.pkl'
    model = load_model(model_path)
    if model is None:
        st.stop()

    with st.form("league_winner_form"):
        matches_played = st.number_input("Matches Played", min_value=0, value=38)
        goals_scored = st.number_input("Goals Scored", min_value=0, value=80)
        goals_conceded = st.number_input("Goals Conceded", min_value=0, value=30)
        target_total_points = st.number_input("Target Total Points", min_value=0, value=90)

        submitted = st.form_submit_button("Predict League Winner")

    if submitted:
        input_data = pd.DataFrame([{
            'matches_played': matches_played,
            'goals_scored': goals_scored,
            'goals_conceded': goals_conceded,
            'target_total_points': target_total_points
        }])
        try:
            prediction = model.predict(input_data)
            prediction_proba = model.predict_proba(input_data)
            result = "Yes, likely to win the league" if prediction[0] == 1 else "No, unlikely to win the league"
            st.success(f"Prediction: {result}")
            st.info(f"Confidence Score (Probability of Winning): {prediction_proba[0][1]:.2%}")
        except Exception as e:
            st.error(f"An error occurred during prediction: {e}")

# --- Model 4: Goals_Assist_model ---
def goals_assists_page():
    st.title("⚽ Player Goals & Assists Prediction")
    st.markdown("Predict player performance (goals/assists) per 90 minutes using player metrics.")

    model_path = 'Goals_Assist_model.pkl'
    model = load_model(model_path)
    if model is None:
        st.stop()

    st.warning("Note: This model requires numeric inputs corresponding to complex per-90 stats. The exact output (goals? assists?) depends on how the model was trained.")

    with st.form("goals_assists_form"):
        col1, col2, col3 = st.columns(3)
        with col1:
            position = st.selectbox("Position", ["DF", "FW", "MF", "GK"])
            matches_played = st.number_input("Matches Played", min_value=0, value=31)
            minutes = st.number_input("Minutes Played", min_value=0, value=2560)
            goals_per_90 = st.number_input("Goals Per 90", min_value=0.0, value=0.04, step=0.01)
            non_penalty_goals_per_90 = st.number_input("Non-Penalty Goals Per 90", min_value=0.0, value=0.40, step=0.01)

        with col2:
            age = st.number_input("Age", min_value=16, max_value=45, value=25)
            starts = st.number_input("Starts", min_value=0, value=30)
            ninetys_played = st.number_input("90s Played", min_value=0.0, value=28.4, step=0.1)
            assists_per_90 = st.number_input("Assists Per 90", min_value=0.0, value=0.04, step=0.01)
            xg_per_90 = st.number_input("xG Per 90", min_value=0.0, value=0.06, step=0.01)

        with col3:
            # Empty column 3 for spacing
            st.markdown("### xAG & npxG")
            xag_per_90 = st.number_input("xAG Per 90", min_value=0.0, value=0.03, step=0.01)
            npxg_per_90 = st.number_input("npxG Per 90", min_value=0.0, value=0.06, step=0.01)


        submitted = st.form_submit_button("Predict Player Performance")

    if submitted:
        # Need to one-hot encode the 'Position' feature before passing to the model
        input_data = pd.DataFrame([{
            "Position": position,
            "Age": age,
            "Matches Played": matches_played,
            "Starts": starts,
            "Minutes": minutes,
            "90s Played": ninetys_played,
            "Goals Per 90": goals_per_90,
            "Assists Per 90": assists_per_90,
            "Non-Penalty Goals Per 90": non_penalty_goals_per_90,
            "xG Per 90": xg_per_90,
            "xAG Per 90": xag_per_90,
            "npxG Per 90": npxg_per_90
        }])

        # Example of simple one-hot encoding for position for prediction
        # (You must ensure this matches how the model was trained)
        positions_df = pd.get_dummies(input_data['Position'], prefix='Position')
        input_data = pd.concat([input_data.drop('Position', axis=1), positions_df], axis=1)

        # Ensure all possible position columns exist with 0 if not present in the input row
        all_positions = ['Position_DF', 'Position_FW', 'Position_MF', 'Position_GK']
        for pos_col in all_positions:
            if pos_col not in input_data.columns:
                input_data[pos_col] = 0

        try:
            # Reorder columns to match the training order if necessary (highly likely)
            # You might need to adjust this part to match your model's exact input requirement
            input_data = input_data[
                ['Age', 'Matches Played', 'Starts', 'Minutes', '90s Played',
                 'Goals Per 90', 'Assists Per 90', 'Non-Penalty Goals Per 90',
                 'xG Per 90', 'xAG Per 90', 'npxG Per 90',
                 'Position_DF', 'Position_FW', 'Position_MF', 'Position_GK']
            ]

            prediction = model.predict(input_data)
            st.success(f"Predicted Output (Goals/Assists depending on model): {prediction[0]:.2f}")
        except Exception as e:
            st.error(f"An error occurred during prediction. Check your model's expected input format/encoding. Error: {e}")

# --- Navigation Setup (Sidebar) ---
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Home", "Match Winner", "Total Team Points", "League Winner", "Goals & Assists Model"])

if page == "Home":
    home_page()
elif page == "Match Winner":
    match_winner_page()
elif page == "Total Team Points":
    total_points_page()
elif page == "League Winner":
    league_winner_page()
elif page == "Goals & Assists Model":
    goals_assists_page()

Writing app.py


In [7]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.3.213:8501

  Stopping...
^C
